In [1]:
import pandas as pd
import numpy as np
import pickle
import gzip
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, MaxPool3D, Dropout, Flatten, Conv3DTranspose, UpSampling3D, Reshape

Using TensorFlow backend.


In [2]:
with open('test_subjects','rb') as f:
               images = pickle.load(f)


#### Subject IDs

In [3]:
im_name_list =[]

for x1 in images:
    im_name_list.append(x1)
im_name_list

['OAS30001',
 'OAS30002',
 'OAS30004',
 'OAS30005',
 'OAS30006',
 'OAS30007',
 'OAS30008',
 'OAS30019',
 'OAS30022',
 'OAS30024',
 'OAS30027']

#### Making training and test sets

In [4]:
pixels=np.zeros((1, 120, 160, 120))
for name in im_name_list:
    pixels= np.append(pixels ,np.reshape(images[name]['image'], (1, 120, 160, 120) ), axis=0)

input_ims= pixels[1:12,:,:,:] 
input_ims.shape

(11, 120, 160, 120)

In [5]:
x_train = input_ims[0:9,:,:,:]
x_train.shape

(9, 120, 160, 120)

In [6]:
x_test=input_ims[9:12,:,:,:]
x_test.shape

(2, 120, 160, 120)

#### Making target values

In [7]:
y_train= [images[key]['group'] for key in images.keys()]
y_test = y_train[-2:]
y_test

['AD', 'AD']

In [8]:
y_train = y_train[:9]
y_train

['CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'AD', 'AD']

In [9]:
def convert_numeric(x):
    if x == "AD":
        return 1
    elif x == "CN":
        return 0
    else:
        return x
    
    
y_train = list(map( convert_numeric, y_train))
y_train

[0, 0, 0, 0, 0, 0, 0, 1, 1]

In [10]:
y_test = list(map( convert_numeric, y_test))
y_test

[1, 1]

In [11]:
len(input_ims)

11

In [12]:
#looking into first image
first_im=images['OAS30001']
first = first_im['image']
first.shape

(120, 160, 120)

In [13]:
first_modified= np.reshape(first, (120, 160, 120 , 1))

#### Building Model

In [14]:
input_ims.shape

(11, 120, 160, 120)

In [15]:
model = Sequential()
#building encoder
reshaped = np.reshape(x_train, (9, 120, 160, 120, 1) )
reshaped.shape

(9, 120, 160, 120, 1)

In [16]:
model.add(Conv3D(10, kernel_size=3, activation='relu', input_shape=(120, 160, 120 , 1), padding='same'))
model.add(MaxPool3D(2))


In [17]:
model.add(Dropout(0.8))

In [18]:
model.add(Conv3D(10, kernel_size=3, activation='relu', padding='same'))

In [19]:
model.add(MaxPool3D(2))

In [20]:
model.add(Dropout(0.8))

In [21]:
model.add(Conv3D(10, kernel_size=3, activation='relu', padding='same'))

In [22]:
model.add(MaxPool3D(2))

In [23]:
model.add(Dropout(0.8))

In [24]:
model.add(Flatten())

In [25]:
model.add(Dense(32, activation='selu'))

In [26]:
model.add(Dense(16, activation='selu'))

In [27]:
model.add(Dense(2, activation='softmax'))

In [28]:
#building decoder

In [29]:
model.add(Dense(16, activation='selu'))

In [30]:
model.add(Dense(32, activation='selu'))

In [31]:
# it may be wrong, but we add one more dense layer to make dimensions match :| (for now) !

In [32]:
model.add(Dense(4500, activation='selu'))

In [33]:
#maybe we need to reshape in here
model.add(Reshape((15, 20, 15, 1)))

In [34]:
model.add(Conv3DTranspose(10, kernel_size=3, activation='relu', padding='same'))

In [35]:
model.add(UpSampling3D(2))

In [36]:
model.add(Dropout(0.8))

In [37]:
model.add(Conv3DTranspose(10, kernel_size=3, activation='relu', padding='same'))

In [38]:
model.add(UpSampling3D(2))

In [39]:
model.add(Dropout(0.8))

In [40]:
model.add(Conv3DTranspose(10, kernel_size=3, activation='relu', padding='same'))

In [41]:
model.add(UpSampling3D(2))

In [42]:
model.add(Dropout(0.8))

In [43]:
model.add(Flatten())

In [44]:
model.compile(optimizer='adam', loss= 'binary_crossentropy')

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 120, 160, 120, 10) 280       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 60, 80, 60, 10)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 80, 60, 10)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 60, 80, 60, 10)    2710      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 30, 40, 30, 10)    0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 40, 30, 10)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 30, 40, 30, 10)   

In [46]:
#Autoencoder compiled sucessfully

In [47]:
#Autoencoder fit the train data

In [48]:
#Build a new model using the encoder part, put the layers in a new model

In [49]:
#check with test data the accuracy and ...